**Transfer Learning model using VGG**

In [1]:
import numpy as np
import time
import torch
import torch.nn as nn

import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import math
import torchvision.models as models


torch.manual_seed(1)

In [ ]:
!unzip 'DatasetAugmented' 


In [4]:
vgg16 = models.vgg.vgg16(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [5]:
data_path = 'DatasetAugmented'

In [6]:
def get_data(len_train_data, len_val_data, len_test_data):


    transform = transforms.Compose([transforms.Resize((224,224)), 
                                    transforms.ToTensor()])

    #Seperate for training, validation, and test data 
    dataset = torchvision.datasets.ImageFolder(data_path, transform=transform)
    num_train = math.floor(len(dataset)*len_train_data)
    num_val = math.floor(len(dataset)*len_val_data)
    num_test = math.floor(len(dataset)*len_test_data)
    print("Number of images for training: ", num_train)
    print("Number of images for validation: ", num_val)
    print("Number of images for validation: ", num_test)
    dummy_len = len(dataset) - (num_test + num_train + num_val)

    # Split into train and validation
    train_set, val_set, test_set, dummy = torch.utils.data.random_split(dataset, [num_train, num_val, num_test, dummy_len]) #80%, 10%, 10% split


    # return train_loader, val_loader, test_loader
    return train_set, val_set, test_set


In [7]:
print("Loading data sets...")
train_data, val_data, test_data = get_data(0.8, 0.2, 0)

classes = ['BlackAugmented', 'AsianAugmented', 'IndianAugmented', 'LatinoAugmented', 'MiddleEasternAugmented', 'WhiteAugmented']

Loading data sets...
Number of images for training:  7192
Number of images for validation:  1798
Number of images for validation:  0


In [8]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [9]:
vgg16.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [14]:
class ANNClassifier(nn.Module):
    def __init__(self):
        super(ANNClassifier, self).__init__()
        self.name = "ANN"
        self.layer1 = nn.Linear(7 * 512 * 7, 10000) 
        self.layer2 = nn.Linear(10000, 1000)
        self.layer3 = nn.Linear(1000, 84)
        self.layer4 = nn.Linear(84, 28)
        self.layer5 = nn.Linear(28, 6) #6 outputs
    def forward(self, img):
        flattened = img.view(-1, 7 * 512 * 7)
        activation1 = self.layer1(flattened)
        activation1 = F.relu(activation1)
        activation2 = self.layer2(activation1)
        activation2 = F.relu(activation2)
        activation3 = self.layer3(activation2)
        activation3 = F.relu(activation3)
        activation4 = self.layer4(activation3)
        activation4 = F.relu(activation4)
        output = self.layer5(activation4)
        output = output.squeeze(1)
        return output



In [11]:
def get_accuracy(model, data, batch_size):
    correct = 0
    total = 0
    for imgs, labels in torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True):

        #############################################
        # To Enable GPU Usage
        if torch.cuda.is_available():
          imgs = imgs.cuda()
          labels = labels.cuda()
        #############################################

        output = model(VGGC(imgs))
        
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [12]:
def train(model, train_data, val_data, learning_rate=0.001, batch_size=64, num_epochs=1):
    torch.manual_seed(1000)  # set the random seed
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

    iters, losses, train_acc, val_acc = [], [], [], []

    # Training
    start_time = time.time()
    n = 0 # the number of iterations
    for epoch in range(num_epochs):
        for imgs, labels in iter(train_loader):
            
            #############################################
            # To Enable GPU Usage
            if torch.cuda.is_available():
              imgs = imgs.cuda()
              labels = labels.cuda()
            #############################################
            
              
            out = model(VGGC(imgs))             # forward pass
            loss = criterion(out, labels) # compute the total loss
            loss.backward()               # backward pass (compute parameter updates)
            optimizer.step()              # make the updates for each parameter
            optimizer.zero_grad()         # a clean up step for PyTorch

            # Save the current training information
            iters.append(n)
            losses.append(float(loss)/batch_size)             # compute *average* loss
            n += 1


        train_acc.append(get_accuracy(model, train_data, batch_size)) # compute training accuracy 
        val_acc.append(get_accuracy(model, val_data, batch_size))  # compute validation accuracy
        print(("Epoch {}: Train acc: {} |"+"Validation acc: {}").format(
                epoch + 1,
                train_acc[-1],
                val_acc[-1]))
        

    print('Finished Training')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

    # Plotting
    plt.title("Training Curve")
    plt.plot(iters, losses, label="Train")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Curve")
    plt.plot(range(1 ,num_epochs+1), train_acc, label="Train")
    plt.plot(range(1 ,num_epochs+1), val_acc, label="Validation")
    plt.xlabel("Epochs")
    plt.ylabel("Training Accuracy")
    plt.legend(loc='best')
    plt.show()

    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))


In [ ]:

model_1 = ANNClassifier()
VGGC = vgg16.features

if torch.cuda.is_available():
    VGGC.cuda()
    model_1.cuda() 
    print('CUDA is available!  Training on GPU ...')
else:
    print('CUDA is not available.  Training on CPU ...')


#torch.cuda.memory_summary(device=None, abbreviated=False)
train(model_1, train_data, val_data, 0.001, 32, 20)

CUDA is available!  Training on GPU ...
Epoch 1: Train acc: 0.17060622914349277 |Validation acc: 0.14905450500556172
Epoch 2: Train acc: 0.35372636262513907 |Validation acc: 0.30589543937708563
Epoch 3: Train acc: 0.4575917686318131 |Validation acc: 0.41101223581757507
Epoch 4: Train acc: 0.5581201334816462 |Validation acc: 0.5083426028921023
Epoch 5: Train acc: 0.6166573971078977 |Validation acc: 0.557285873192436
Epoch 6: Train acc: 0.692853170189099 |Validation acc: 0.5884315906562848
Epoch 7: Train acc: 0.803392658509455 |Validation acc: 0.6340378197997776
Epoch 8: Train acc: 0.8520578420467185 |Validation acc: 0.6434927697441601
Epoch 9: Train acc: 0.9491101223581757 |Validation acc: 0.6907675194660734


In [ ]:
model_2 = ANNClassifier()
VGGC = vgg16.features

if torch.cuda.is_available():
    VGGC.cuda()
    model_2.cuda() 
    print('CUDA is available!  Training on GPU ...')
else:
    print('CUDA is not available.  Training on CPU ...')


torch.cuda.memory_summary(device=None, abbreviated=False)
train(model_2, train_data, val_data, 0.0001, 32, 15)